In [1]:
import tensorflow as tf
gpu_options = tf.GPUOptions(allow_growth=True, per_process_gpu_memory_fraction=0.1)
s = tf.InteractiveSession(config=tf.ConfigProto(gpu_options=gpu_options))

In [2]:
# s.run(<your computation>)
# with s:
#     <>.eval()

In [3]:
import numpy as np

In [4]:
def sum_squares(N):
    return np.sum(np.arange(N)** 2)

In [5]:
%%time
sum_squares(10**8)

CPU times: user 293 ms, sys: 333 ms, total: 626 ms
Wall time: 624 ms


662921401752298880

In [6]:
with tf.device('/cpu:0'):
    N = tf.placeholder('int64', name="input_to_your_function")
    result = tf.reduce_sum((tf.range(N)**2))

In [7]:
%%time
print(result.eval({N:10**8}))

662921401752298880
CPU times: user 1.01 s, sys: 169 ms, total: 1.18 s
Wall time: 256 ms


In [8]:
aribitrary_input = tf.placeholder('float32')
input_vector = tf.placeholder('float32',shape=(10,))

In [9]:
input_matrix  =tf.placeholder('float32',shape=(None,15))

In [10]:
double_the_vector = input_vector*2

In [11]:
elementwise_cosine = tf.cos(input_vector)

In [12]:
elementwise_cosine

<tf.Tensor 'Cos:0' shape=(10,) dtype=float32>

In [13]:
v1 = tf.placeholder(dtype='float32',shape=(None,))
v2 = tf.placeholder(dtype='float32',shape=(None,))

trafo = (v1*v2)/(tf.sin(v1)+1)

In [14]:
trafo

<tf.Tensor 'truediv:0' shape=(?,) dtype=float32>

In [15]:
dummy = np.arange(5).astype('float32')

In [16]:
trafo.eval({v1:dummy, v2:dummy[::-1]})

array([0.       , 1.6291324, 2.0950115, 2.6289961, 0.       ],
      dtype=float32)

In [17]:
dummy

array([0., 1., 2., 3., 4.], dtype=float32)

In [18]:
dummy[::-1]

array([4., 3., 2., 1., 0.], dtype=float32)

In [21]:
import os
os.environ['USER']

# import os
# port = 6000 + os.getuid()
# print("Port: %d" % port)
# #!killall tensorboard
# os.system("tensorboard --logdir=./tboard --port=%d &" % port)

# # show graph to tensorboard
# writer = tf.summary.FileWriter("./tboard", graph=tf.get_default_graph())
# writer.close()


'thea.aarrestad'

In [22]:
logdir = os.path.join('/mnt/students-home',os.environ['USER'],'tensorflow-logs')
writer = tf.summary.FileWriter(logdir, graph=tf.get_default_graph())
writer.close()

In [ ]:
mse = tf.reduce_sum( (v1-v2)**2 )

In [ ]:
mse

In [ ]:
f = lambda a,b: a+b

In [ ]:
f

In [ ]:
f(1,2)

In [ ]:
compute_mse = lambda vec1,vec2 : mse.eval( {v1: vec1, v2:vec2} )

In [ ]:
dummy1 = np.arange(6).astype('float32')
compute_mse(dummy1,dummy1[::-1])

In [ ]:
# Creating shared variables
shared_vec1 = tf.Variable(initial_value=np.ones(5),dtype='float32')
s.run(tf.global_variables_initializer())
print("initial value", s.run(shared_vec1))

In [ ]:
s.run(shared_vec1.assign(np.arange(5)))
print("new value", s.run(shared_vec1))

In [ ]:
my_scalar = tf.placeholder('float32')
scalar_squared = my_scalar**2
derivative = tf.gradients(scalar_squared,my_scalar)[0]

In [ ]:
x

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

x = np.linspace(-3,3)
x_squared, x_squared_der = s.run([scalar_squared,derivative], {my_scalar:x})

plt.plot(x, x_squared,label="x^2")
plt.plot(x, x_squared_der, label="derivative")
plt.legend();

In [ ]:
my_vector = tf.placeholder('float32',[None])

weird_psychotic_function = tf.reduce_mean((my_vector+my_scalar)**(1+tf.nn.moments(my_vector,[0])[1]) + 1./ tf.atan(my_scalar))/(my_scalar**2 + 1) + 0.01*tf.sin(2*my_scalar**1.5)*(tf.reduce_sum(my_vector)* my_scalar**2)*tf.exp((my_scalar-4)**2)/(1+tf.exp((my_scalar-4)**2))*(1.-(tf.exp(-(my_scalar-4)**2))/(1+tf.exp(-(my_scalar-4)**2)))**2

der_by_scalar = tf.gradients(weird_psychotic_function,my_scalar)
der_by_vector = tf.gradients(weird_psychotic_function,my_vector)

In [ ]:
der_by_vector

In [ ]:
#Plotting your derivative
scalar_space = np.linspace(1, 7, 100)

y = [s.run(weird_psychotic_function, {my_scalar:x, my_vector:[1, 2, 3]})
     for x in scalar_space]

plt.plot(scalar_space, y, label='function')

y_der_by_scalar = [s.run(der_by_scalar, {my_scalar:x, my_vector:[1, 2, 3]})
     for x in scalar_space]

plt.plot(scalar_space, y_der_by_scalar, label='derivative')
plt.grid()
plt.legend();

In [ ]:
y_guess = tf.Variable(np.zeros(2,dtype='float32')) #unknown variable, not fixed
y_true = tf.range(1,3,dtype='float32')

loss = tf.reduce_mean((y_guess - y_true + tf.random_normal([2]))**2) 
optimizer = tf.train.MomentumOptimizer(0.01,0.9).minimize(loss,var_list=y_guess)

#same, but more detailed:
#updates = [[tf.gradients(loss,y_guess)[0], y_guess]]
#optimizer = tf.train.MomentumOptimizer(0.01,0.9).apply_gradients(updates)
#y_guess = y_guess - 0.01*updates

In [ ]:
from IPython.display import clear_output

s.run(tf.global_variables_initializer())

guesses = [s.run(y_guess)]

for _ in range(200):
    s.run(optimizer)
    guesses.append(s.run(y_guess))
    
    clear_output(True)
    plt.plot(*zip(*guesses),marker='.')
    plt.scatter(*s.run(y_true),c='red')
    plt.show()

In [23]:
from sklearn.datasets import load_digits
mnist = load_digits(2)

X,y = mnist.data, mnist.target

print("y [shape - %s]:" % (str(y.shape)), y[:10])
print("X [shape - %s]:" % (str(X.shape)))

y [shape - (360,)]: [0 1 0 1 0 1 0 0 1 1]
X [shape - (360, 64)]:


In [24]:
print('X:\n',X[:3,:10])
print('y:\n',y[:10])
plt.imshow(X[0].reshape([8,8]))

X:
 [[ 0.  0.  5. 13.  9.  1.  0.  0.  0.  0.]
 [ 0.  0.  0. 12. 13.  5.  0.  0.  0.  0.]
 [ 0.  0.  1.  9. 15. 11.  0.  0.  0.  0.]]
y:
 [0 1 0 1 0 1 0 0 1 1]


NameError: name 'plt' is not defined

In [ ]:
# y_pred   = 1./ (1+exp(-dot(w,x))
# xentropy = sum[-y * log(y_pred)+(1-y)*log(1-y_pred)

In [ ]:
mean_X = np.mean(X,axis=0)
std_X = np.std(X,axis=0)
X = (X-mean_X)/(std_X + 1e-7)

weights = tf.Variable(dtype='float32',expected_shape=(64,),name='logreg_w',initial_value=np.random.normal(size=64))
bias    = tf.Variable(dtype='float32',expected_shape=(1,) ,name='logreg_b' ,initial_value=np.random.normal(size=1))
input_X = tf.placeholder(dtype='float32',shape=(None,64))
input_y = tf.placeholder(dtype='float32',shape=(None,1))

In [ ]:
predicted_y = tf.reduce_sum(tf.multiply(input_X,weights), axis=1)+bias # [batch_size,64*64+[1]-->[batch_size,64] for scalar
loss = -tf.reduce_mean(input_y*tf.log(predicted_y+1e-6) + (1-input_y)*tf.log(1-predicted_y*1e-6))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01).minimize(loss,var_list=[weights,bias])

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [ ]:
s.run(tf.global_variables_initializer())

In [ ]:
from sklearn.metrics import roc_auc_score

for i in range(5):
    
    s.run(optimizer,feed_dict={input_X: X_train,
                               input_y:y_train})
    
    loss_i_train = s.run(loss,feed_dict={input_X: X_train, 
                                         input_y:y_train})
    loss_i_test = s.run(loss,feed_dict={input_X: X_test, 
                                        input_y:y_test})
    
    print("train loss at iter %i:%.4f" % (i, loss_i_train))
    print("test loss at iter %i:%.4f" % (i, loss_i_test))
    
#     print("train auc:",roc_auc_score(y_train, predict_function(X_train)))
#     print("test auc:",roc_auc_score(y_test, predict_function(X_test)))

    
# print ("resulting weights:")
# plt.imshow(shared_weights.get_value().reshape(8, -1))
# plt.colorbar();